## 🔑 Part 1 연습문제 정답지

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import r2_score

In [ ]:
# 데이터 로드
# 보스턴 주택 가격 데이터셋 생성 (실제 보스턴 데이터셋 대신 시뮬레이션)
np.random.seed(42)
n_samples = 506

# 특성 생성
data = {
    'LotArea': np.random.normal(10000, 3000, n_samples),
    'OverallQual': np.random.randint(1, 11, n_samples),
    'BldgType': np.random.choice(['1Fam', '2fmCon', 'Duplex', 'TwnhsE', 'Twnhs'], n_samples),
    'Neighborhood': np.random.choice(['CollgCr', 'Veenker', 'Crawfor', 'NoRidge', 'Mitchel', 'Somerst', 'NWAmes'], n_samples)
}

# 타겟 변수 생성 (주택 가격)
price = (data['LotArea'] * 0.01 + 
         data['OverallQual'] * 15000 + 
         np.random.normal(0, 10000, n_samples))

df = pd.DataFrame(data)
df['SalePrice'] = np.maximum(price, 50000)  # 최소 가격 설정

# 일부 결측치 추가
missing_indices = np.random.choice(df.index, size=int(0.1 * len(df)), replace=False)
df.loc[missing_indices[:len(missing_indices)//2], 'LotArea'] = np.nan
df.loc[missing_indices[len(missing_indices)//2:], 'BldgType'] = np.nan

print("데이터셋 정보:")
print(df.info())
print("\n처음 5행:")
print(df.head())

# 특성과 타겟 분리
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\n훈련 데이터 크기: {X_train.shape}")
print(f"테스트 데이터 크기: {X_test.shape}")

# 기본 전처리 파이프라인 설정
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# 기본 모델 파이프라인
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                ('regressor', LinearRegression())])

print("\n기본 파이프라인이 설정되었습니다.")


In [ ]:
numeric_features = ['LotArea', 'OverallQual']
categorical_features = ['BldgType', 'Neighborhood']


### 1.3 연습 문제 정답 보기

1.  **Imputer 전략 변경**

In [ ]:
# 수치형: median
numeric_transformer_median = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# 범주형: most_frequent
categorical_transformer_freq = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor_new = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_median, numeric_features),
        ('cat', categorical_transformer_freq, categorical_features)])

model_pipeline_new = Pipeline(steps=[('preprocessor', preprocessor_new),
                                      ('regressor', LinearRegression())])
model_pipeline_new.fit(X_train, y_train)
# 이후 예측 및 평가는 동일

2.  **Ridge 모델 파이프라인**

In [ ]:
from sklearn.linear_model import Ridge

ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), # 기존 preprocessor 재사용
    ('regressor', Ridge(alpha=1.0)) # Ridge 모델로 교체
])
ridge_pipeline.fit(X_train, y_train)


### 2.3 연습 문제 정답 보기

1.  **LSTAT 변수 사용**

In [ ]:
# Ames Housing 데이터셋의 실제 컬럼명 사용
X_lstat = df[['LotArea']]  # 또는 적절한 면적 관련 변수
y = df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X_lstat, y, test_size=0.3, random_state=42)

lr_lstat = LinearRegression().fit(X_train, y_train)
y_pred_lstat = lr_lstat.predict(X_test)
print(f"LotArea 모델 R-squared: {r2_score(y_test, y_pred_lstat):.4f}")
# GrLivArea 모델의 R-squared와 비교하여 성능을 확인할 수 있습니다.
# 실제 Ames Housing 데이터셋에서는 GrLivArea가 더 강한 예측력을 보일 가능성이 높습니다.

2.  **LSTAT 계수 해석**
    계수는 음수가 나옵니다 (`lr_lstat.coef_[0]` 확인 시 약 -0.9 \~ -1.0). 이는 저소득층 비율(`LSTAT`)이 높을수록 주택 가격(`SalePrice`)은 낮아지는 경향이 있음을 의미하며, 이는 상식적으로 타당한 해석입니다.

### 3.3 연습 문제 정답 보기

1.  **NOX 변수 추가**

In [ ]:
features_new = ['GrLivArea', 'CRIM', 'PTRATIO', 'BsmtFinSF1']
X_multi_new = df[features_new]
# ... (이하 train_test_split, fit, predict, score 과정은 동일) ...
# R-squared 값이 기존보다 소폭 상승하는 것을 확인할 수 있습니다.

2.  **계수 값이 변하는 이유**
    다중 회귀에서 각 변수의 계수는 **'다른 변수들이 통제(고정)되었을 때'** 의 순수한 영향력을 의미합니다. 
    
    단순 회귀에서의 `GrLivArea` 계수는 다른 변수(범죄율, 학생-교사 비율 등)와 `GrLivArea` 간의 숨겨진 관계까지 모두 포함된 값이었습니다. 
    
    예를 들어, 방 개수가 많은 지역은 범죄율이 낮은 경향이 있을 수 있습니다. 
    
    다중 회귀에서는 이러한 다른 변수들의 효과를 분리하여 `GrLivArea`만의 영향력을 추정하기 때문에 계수 값이 달라집니다.


### 4.3 연습 문제 정답 보기

1.  **Degree 3 모델 비교**

In [ ]:
# Degree 3 모델 생성
poly_reg_3 = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
poly_reg_3.fit(X_train, y_train)
test_rmse_3 = np.sqrt(mean_squared_error(y_test, poly_reg_3.predict(X_test)))
print(f"Degree 3 Test RMSE: {test_rmse_3:.2f}")
# Degree 2 모델보다 Test RMSE가 약간 낮아져 성능이 개선될 수 있습니다.
# 하지만 Degree 10 보다는 훨씬 좋은 성능을 보입니다.

2.  **차수별 RMSE 그래프**

In [ ]:
train_rmse_list = []
test_rmse_list = []
degree_range = range(1, 16)

for degree in degree_range:
    model = make_pipeline(PolynomialFeatures(degree=degree), LinearRegression())
    model.fit(X_train, y_train)

    train_rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
    test_rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

    train_rmse_list.append(train_rmse)
    test_rmse_list.append(test_rmse)

# Plotly로 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(degree_range), y=train_rmse_list, mode='lines+markers', name='Train RMSE'))
fig.add_trace(go.Scatter(x=list(degree_range), y=test_rmse_list, mode='lines+markers', name='Test RMSE'))
fig.update_layout(title='차수에 따른 RMSE 변화', xaxis_title='Degree', yaxis_title='RMSE', template='plotly_white')
fig.show()
# 그래프를 보면, Train RMSE는 차수가 높아질수록 계속 감소하지만,
# Test RMSE는 특정 지점(대략 3~5)에서 최저점을 찍고 다시 증가하는 것을 볼 수 있습니다.
# 이 지점이 과적합이 시작되는 구간입니다.